#🌍globale Parameter



In [1]:
### HYPERPARAMETER ###

# START_URL = 'https://www.ewms-tech.com/'
START_URL = "https://www.rue-zahnspange.de/"
# START_URL = 'https://www.malerarbeiten-koenig.de/'


EXCLUDED_WEBSITES = ["impressum", "datenschutz", "datenschutzerklärung", "agb"]



from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

PROJECT_ROOT = userdata.get("gdrive_seo_root")
PROJECT_ROOT_ESC_STR = PROJECT_ROOT.replace('Colab Notebooks', 'Colab\ Notebooks')

SRC_PATH, DATA_PATH, TEST_PATH, OUTPUT_PATH = PROJECT_ROOT + "/src", PROJECT_ROOT + "/data", PROJECT_ROOT + '/tests', PROJECT_ROOT + '/output'
FAISS_PATH = DATA_PATH + '/faiss_db'
KEYWORD_PATH = DATA_PATH + '/keywords'
PROMPT_PATH = DATA_PATH + '/prompts'


FINAL_IMAGES = {}


Mounted at /content/drive


# 📔 READ_ME und TO_DO

#### README

In [2]:
%%writefile '/content/drive/MyDrive/Colab Notebooks/SEO/README.md'
# 🚀 SEO Automation Pipeline mit OpenAI & Retrieval (RAG)

Dieses Projekt bietet eine **komplette End-to-End-Pipelines für die SEO-Optimierung von Websites**, inklusive **Web-Crawling, SEO-Analyse, KI-gestützter Text-Optimierung und Qualitätskontrolle**.

Kern des Projekts sind **automatisierte Abläufe**, die von der **Datengewinnung bis zur SEO-optimierten Textgenerierung** reichen.
Es wird eine pipeline zur Text-Optimierung Mithilfe von **OpenAI (ChatGPT)** hergestellt. Die Textelemente eines HTML Dokumentes werden mit ChatGPT SEO optimiert und danach wieder in den code eingebaut.
Der Kunde erhält eine Datei als Vorschau auf seine SEO optimierte website.

## 📚 Inhaltsverzeichnis

- Features
- Projektstruktur
- Ablauf & Module
- Technologien
- Installation
- Nutzung
- Ziele
- Roadmap

## ✅ Features

- 🌐 **Automatisiertes Web Crawling** (inkl. Filter für relevante Inhalte)
- ✍️ **Generierung von SEO-optimierten Texten** mithilfe der OpenAI API
- 🧠 **RAG-gestützte Fehlererkennung & Textkorrektur** mit Vektordatenbank (FAISS)
- 📊 **Analyse der Optimierungsergebnisse** (Statistiken, Ähnlichkeiten, Visualisierungen)
- 📈 **Keyword-Analyse und Keyword-Optimierung**
- 📦 Ausgabe in **HTML und PDF** für Kunden
- 📊 Umfangreiche **Datenvisualisierungen** (Wordclouds, Cosine Similarity, Keyword-Verteilung)




<img src="https://drive.google.com/uc?id=10oR2bcugvN2MClp14ia7gnzMGX5b896t" alt="SEO Heatmap" width="600">




## 🗂️ Projektstruktur

```
SEO-Project/
├── data/                # Prompts, Fehler-Korrektur-Daten, weitere JSON Dateien
├── notebooks/           # Colab/Notebooks zum Starten und Entwickeln
├── output/              # Erzeugte Dateien (HTML, PDF, Bilder)
│   ├── final           # Dokumente für Kunden (HTML, PDF)
│   └── images          # Visualisierungen
├── src/                # Source Code (Python-Klassen und Module)
│   ├── webscraper.py    # Webscrawling und Text-Extraktion
│   ├── llmprocessor.py # Anbindung an OpenAI API, Keyword Extraktion
│   ├── chatbot.py       # Zentrale Chatbot-Klasse zur Kommunikation mit GPT
│   ├── seoanalyzer.py   # Analyse und Auswertung der Texte
│   ├── github.py        # Automatischer Upload ins GitHub Repo
│   ├── utils.py         # Hilfsmodule (z.B. für Prompt-Management)
│   └── embeddingdemo.py# 3D Embedding- und Cosine Similarity Visualisierungen
├── tests/              # pytest der Hauptfunktionalitäten
└── requirements.txt    # Python-Abhängigkeiten
```

## ⚙️ Ablauf & Module

### 1. **Web Crawling**
- **src/webscraper.py**: Holt Inhalte von Webseiten, filtert irrelevante Seiten (z.B. Impressum, AGB).

### 2. **SEO-Optimierung mit OpenAI**
- **src/llmprocessor.py**:
  - Extrahiert Keywords aus den Inhalten.
  - Optimiert die Texte für SEO mit gezielten Prompts.

### 3. **Analyse & Visualisierung**
- **src/seoanalyzer.py**: Verarbeitet und analysiert die Original- und optimierten Texte.

### 4. **GitHub Automation**
- **src/github.py**: Lädt finale Ergebnisse in ein GitHub-Repo hoch.

## 🧰 Technologien

| Technologie                  | Beschreibung                                       |
|-----------------------------|---------------------------------------------------|
| Python                      | Hauptsprache                                       |
| OpenAI API (ChatGPT, GPT-4)  | Generative KI für SEO-Texte                       |
| FAISS                      | Vektorsuche für RAG und Text-Fehler                |
| Pandas, NumPy               | Datenanalyse und Verarbeitung                      |
| Matplotlib, Seaborn         | Visualisierungen                                   |
| Sentence Transformers       | Embedding-Erstellung für Vektordatenbank          |
| BeautifulSoup, Requests     | Webcrawling                                        |
| Google Colab                | Entwicklung und Ausführung                        |

## 🚀 Installation

```bash
pip install -r requirements.txt
python -m spacy download de_core_news_sm
pip install faiss-cpu sentence-transformers openai wordcloud matplotlib seaborn
```

## 💻 Nutzung

```python
scraper = WebsiteScraper(start_url="https://www.example.com")
scraper.scrape_website()

llm_processor = LLMProcessor(prompts_folder, get_filtered_texts, google_ads_keywords)
llm_processor.run_all()

seo_analyzer = SEOAnalyzer(seo_json, original_texts, keywords_final)
seo_analyzer.run_analysis()
```

## 🎯 Ziele

- ✅ Vollständige Automatisierung der SEO-Optimierung
- ✅ RAG für sprachliche Qualitätskontrolle
- ✅ Kundenfertige PDF/HTML-Reports

## 🚧 Roadmap

- [ ] **Produkt für Kunden finalisieren:** all-in-one solution für webcrawl + SEO + optimierten html code
- [ ] Automatische SEO Scores (z.B. Google Ads API)
- [ ] Automatische Keyword-Erweiterung
- [ ] Mehrsprachigkeit
- [ ] WordPress-Integration


Overwriting /content/drive/MyDrive/Colab Notebooks/SEO/README.md


####TODO

In [3]:
%%writefile '/content/drive/MyDrive/Colab Notebooks/SEO/TODO.md'
# To-Do Liste: SEO Automation & KI-Projekt

Diese Liste fasst alle anstehenden Aufgaben im Projekt zusammen

---

## 0. **Aktuelles und dringendes**
- [ ] 18.3.2025 **Version missmatch**: numpy 2.2.3 und spacy 3.5 **side effects on**: dependencies.py, excelimporter.py, Installation.ipynb **ursachen**: spacy version 3.5 verlangt numpy 1.26.4 -> version missmatch
      -> 23.5. class SEOAnlyzer refaktoriert und spacy ersetzt
- [ ] 28.3.25 lokale SEO keywords liefern manchmal falsche Stadt
- [ ] prompts besser organisieren
- [ ] all-in-one lösung weiter verfolgen

---

## 1. **Allgemeine Projektorganisation**
- [ ] **Projektstruktur verbessern**: Ordner übersichtlich gestalten (z.B. `src/`, `data/`, `tests/`, `notebooks/`, dependencies.py).
- [ ] **Dokumentation erweitern**: READ_ME und Wiki (bzw. GitHub Pages) zu jedem Modul anlegen.
- [ ] **Automatisierte Tests** Pytest für Kernfunktionen ausbauen.
- [ ] **Produkt für Kunden finalisieren**
- [ ] **FAISS DB**: automatisierte Erweiterung bei neu gefundenen Fehlern
- [ ] **Template GitHub**: issues
- [ ] Funktionalitäten aus **utils.py** überdenken
- [ ] langfristig Umstieg auf **langchain**
- [ ] textprocessor durch openai **function calling** ersetzen
- [ ] **dependencies** und versionen robuster machen
- [ ] **bug reporting system** einrichten

---

## 2. **Vector-Datenbank (FAISS) & Retrieval**
- [ ] **VectorDB-Klasse finalisieren**:
  - [ ] Kleinere Bugs beheben
  - [ ] Userfreundliche Methoden für neue Einträge
- [ ] **Einrichtung der DB** bei Projektstart (Neubau vs. Laden) vereinheitlichen
- [ ] **Konfigurierbare Ähnlichkeits-Schwelle** (z.B. `threshold=0.6`) besser dokumentieren
- [ ] **Dynamische Filter** für bestimmte Fehlerkategorien (z.B. Stil vs. Grammatik) überlegen
- [ ] **hybrides system mit knowledge tree und RAG** etablieren

---

## 3. **SEO-Optimierungs-Pipeline**
- [ ] **Prompts in JSON-Dateien** verlagern (z.B. `/data/prompts/`) und sauber verlinken
- [ ] **Supervisor-Feedback** integrieren & QA-Schritte definieren
- [ ] Langchain und SEOPageOptimizer verbinden

---

## 4. **SEOGrammarChecker & PromptManager**
- [ ] Klassenrefactoring:
  - [ ] **`VectorDB`** vs. **`PromptManager`** vs. **`SEOGrammarChecker`** sauber trennen
  - [ ] Möglichst wenig Code-Duplikate, mehr modulare Testbarkeit
- [ ] **Konfigurationsdatei** (z.B. YAML) für Pfade, wie `FAISS_PATH` & Promptordner
- [ ] **Erweiterbare Prompt-Templates**:
  - [ ] Z.B. `seo_optimization.json`, `grammar_check.json`, `supervisor.json`, etc.

---

## 5. **Abschluss & Integration**
- [ ] **Dokumentation** aller Pipelines & Klassen in der README (oder in separater Doku)
- [ ] **Optionale WordPress-Integration** in der Zukunft (Ideenspeicher)
  - [ ] Upload via REST API
  - [ ] Metadaten (Title, Slug, Tags etc.)


Overwriting /content/drive/MyDrive/Colab Notebooks/SEO/TODO.md


# 🏁 Install requirements

In [4]:
%run '/content/drive/MyDrive/Colab Notebooks/SEO/notebooks/Installation.ipynb'

In [5]:
# %run '/content/drive/MyDrive/Colab Notebooks/SEO/src/dependencies.py'

# ⛩ push to github

In [6]:
import importlib
import github
importlib.reload(github)
from github import GitHubManager

# Starte den GitHub-Sync
git_manager = GitHubManager(
    userdata.get("github_pat"),
    userdata.get("github_email"),
    userdata.get("github_username"),
    userdata.get("github_repo_seo"),
    PROJECT_ROOT_ESC_STR
)

git_manager.clone_repo()  # Klonen des Repos
git_manager.sync_project()

📥 Klonen des GitHub-Repositories...
📂 Kopiere das Projekt ins Repository...
🔧 Git-Konfiguration wird gesetzt...
➕ Änderungen hinzufügen...
📌 Änderungen committen...
⬆️ Änderungen werden auf GitHub gepusht...
✅ Repository erfolgreich synchronisiert!


# 🕸 crawl

In [7]:
import importlib
import webscraper
importlib.reload(webscraper)
from webscraper import WebsiteScraper

scraper = WebsiteScraper(start_url=START_URL, max_pages=20, excluded_keywords=EXCLUDED_WEBSITES)

original_texts = scraper.get_filtered_texts()

# 🔑 SEO keywords

### 📺 google ads seo keywords

In [8]:
import excelimporter
importlib.reload(excelimporter)
from excelimporter import ExcelImporter

importer = ExcelImporter(project_folder=PROJECT_ROOT, header=2)
keyword_df = importer.import_all()

if keyword_df is not None:
    excluded_seo_keywords = ['spange de', 'kfo zentrum essen', 'gerade zahne', 'zahn spange']

    keyword_df = keyword_df[(keyword_df['Avg. monthly searches'] > 10) &
    (~keyword_df['Keyword'].isin(excluded_seo_keywords))
    ].sort_values(by='Avg. monthly searches', ascending=False).reset_index(drop=True).copy()

    google_ads_keywords = list(keyword_df['Keyword'])

    keyword_df.set_index(keyword_df.columns[0], inplace=True)
    searches_df = keyword_df[[col for col in keyword_df.columns if col.startswith('Searches')]]

else:
    print("Error: importer.import_all() returned None. Check your data file and ExcelImporter class.")
    google_ads_keywords = None

✅ Geladen: /content/drive/MyDrive/Colab Notebooks/SEO/data/keywords/Keyword Stats 2025-03-09 at 12_41_38.xlsx


In [9]:
import keywordvisualizer
importlib.reload(keywordvisualizer)
from keywordvisualizer import KeywordVisualizer


visualizer = KeywordVisualizer(
    df=searches_df,
    output_dir=OUTPUT_PATH+'/images',
    image_paths=FINAL_IMAGES
)
visualizer.heatmap()

ℹ️ Logarithmische Skalierung aktiviert.
✅ Heatmap gespeichert als: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png


'/content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png'

##🔮keywords LLM

In [10]:
import chatbot
importlib.reload(chatbot)
from chatbot import Chatbot
from utils import load_prompts

prompts = load_prompts(PROMPT_PATH + '/extract_keywords.json')
system_prompt = prompts['system_prompt']
user_prompt = prompts['user_prompt']

In [11]:
user_prompt = user_prompt.replace('{original_text}', " ".join(list(original_texts.values())))

In [12]:
ch = Chatbot(system_prompt, user_prompt)
keywords_llm = ch.chat()

In [13]:
keywords_llm_google = keywords_llm.split(", ") + google_ads_keywords

# 🛏 embedding demo

In [14]:
import os
import embeddingdemo
importlib.reload(embeddingdemo)
from embeddingdemo import EmbeddingDemo

demo = EmbeddingDemo(output_dir=OUTPUT_PATH+'/images', final_images=FINAL_IMAGES)
demo.run_all_visualizations()

print("Alle Embedding-Bilder:", demo.get_image_paths())
print("Gemeinsame Bilder (shared dict):", FINAL_IMAGES)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


📊 3D Keyword Similarity
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/embedding_keyword_3d_2025.png

🔍 Cosine Similarity Steps
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_similarity_steps_2025.png

📍 Vergleich als Vektoren (3D)
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_comparison_3d_2025.png

🧠 BERT Embeddings Visualisierung (plotly)
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/bert_embeddings_3d_2025.png / /content/drive/MyDrive/Colab Notebooks/SEO/output/images/bert_embeddings_3d_2025.svg
Alle Embedding-Bilder: {'keyword_heatmap': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png', 'embedding_keyword_3d': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/embedding_keyword_3d_2025.png', 'cosine_similarity_steps': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_similarity_steps_2025.png', 'cos

# 📥 Tests

#### 📃 Doku

In [15]:
%%writefile '/content/drive/MyDrive/Colab Notebooks/SEO/tests/DOKU_TESTS.md'
# ✅ Pytest Template: Chatbot Klasse

## 1. Klassen & Methoden die getestet werden sollen

- **Chatbot**
  - `chat()`
  - `chat_with_streaming()`

---

## 2. Beispielhafte Inputs + erwartete Outputs pro Methode

| Methode                      | Beispiel Input                                                           | Erwartete Ausgabe    |
|-----------------------------|-------------------------------------------------------------------------|----------------------|
| `Chatbot.chat()`             | 'Das ist ein Test. Schreibe als Antwort "Test erfolgreich".'           | "Test erfolgreich"   |
| `Chatbot.chat_with_streaming()` | 'Das ist ein Test. Schreibe als Antwort "Test erfolgreich".'         | "Test erfolgreich"   |

---

## 3. Return-Typen der Methoden

| Methode                      | Rückgabe-Typ |
|-----------------------------|--------------|
| `Chatbot.chat()`             | `str`        |
| `Chatbot.chat_with_streaming()` | `str`     |

---

## 4. Externe Services mocken?

| Service         |  Mocken?                           |
|-----------------|-------------------------------------|
| OpenAI API      |  Nein                                |
| FAISS Index     |  Ja (kleine Test-Datenbank für FAISS) |

---

## 5. Ordnerstruktur für Tests

```bash
/project-root/
    /src/
        chatbot.py
    /tests/
        test_chatbot.py
    /logs/
        test_report.log
    ...
```

---


Overwriting /content/drive/MyDrive/Colab Notebooks/SEO/tests/DOKU_TESTS.md


#### 👷 code

In [16]:
import pytest
pytest.main(['-v', TEST_PATH+'/chatbot_test/chatbot_test.py'])

============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: langsmith-0.3.28, anyio-4.9.0, typeguard-4.4.2
collecting ... collected 2 items

drive/MyDrive/Colab Notebooks/SEO/tests/chatbot_test/chatbot_test.py::test_chat_method PASSED [ 50%]
drive/MyDrive/Colab Notebooks/SEO/tests/chatbot_test/chatbot_test.py::test_chat_with_streaming_method PASSED [100%]

=============================== warnings summary ===============================
../usr/local/lib/python3.11/dist-packages/_pytest/config/__init__.py:1277
  /usr/local/lib/python3.11/dist-packages/_pytest/config/__init__.py:1277: PytestAssertRewriteWarning:
  
  Module already imported so cannot be rewritten: anyio

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 2 passed, 1 warning in 1.76s =========================


<ExitCode.OK: 0>

# 🎨 Error Collection

In [17]:
error_corrections = {
    "Eine Zahnspange kann Kiefergelenksbeschwerden, Kauen- und Sprechprobleme effektiv behandeln.":
    "Eine Zahnspange kann Kiefergelenksbeschwerden, sowie Kau- und Sprechprobleme effektiv behandeln.",
    "Als in den USA geborene Kieferorthopädin bringt Dr. Meier eine multikulturelle Perspektive mit und spricht neben Deutsch auch Englisch, Swahili sowie über Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch.":
    "Als in den USA geborene Kieferorthopädin bringt Dr. Meier eine multikulturelle Perspektive mit und spricht neben Deutsch auch Englisch und Swahili. Dazu hat sie Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch.",
    "Sie hat ihren Master of Science in Geochemie von der Universität Münster, Deutschland, und hat an der Universität Düsseldorf abgeschlossen.":
    "Sie hat ihren Master of Science in Geochemie von der Universität Münster, Deutschland, und hat an der Universität Düsseldorf promoviert.",
    "Ihre Qualifikationen umfassen nicht nur Fachwissen, sondern auch eine besondere Hingabe zu einem ästhetischen Lächeln.":
    "Sie ist hoch qualifiziert und hat eine besondere Hingabe zu einem ästhetischen Lächeln.",
    "behandlungsorientierte Zahnberatung": "patientenorientierte Beratung",
    "ästehthetisches Lächeln": "ästhetisches Lächeln",
    "Nachdem Ihr Behandlungsplan von der Krankenkasse genehmigt wurde": "Nachdem Ihr Behandlungsplan von der Krankenkasse bestätigt wurde",
    "Der aktuelle Text zur Zahnspangenpraxis": "Der aktuelle Text zur kieferorthopädischen Praxis"
}

In [18]:
new_error_corrections = {"Das ist ein neuer Fehler.": "Das ist ein korrigierter Fehler."}

# 🎨 RAG

In [19]:
%%capture
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from chatbot import Chatbot

# -------------------------
# 1) VectorDB
# -------------------------

#test_text = seo_json[list(seo_json.keys())[0]]["SEO"]

class VectorDB:
    """
    Eine Klasse für alles rund um die Vektordatenbank:
    - Aufbauen & Laden (FAISS)
    - Neue Einträge hinzufügen
    - Querying für Context Retrieval
    """

    def __init__(self, db_folder):
        """
        :param db_folder: Pfad zum Datenbank-Ordner
        """
        self.db_folder = db_folder
        self.index_file = os.path.join(db_folder, "faiss_index.bin")
        self.json_file  = os.path.join(db_folder, "faiss_index.json")

        self.index = None
        self.error_dict = {}

        self.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def build_index(self, error_corrections: dict):
        """
        Baut einen neuen FAISS-Index aus den übergebenen Fehler-Korrektur-Paaren.
        """
        print("🔨 Baue neuen FAISS-Index...")
        os.makedirs(self.db_folder, exist_ok=True)

        self.error_dict = error_corrections
        errors = list(self.error_dict.keys())

        # Embeddings
        embeddings = np.array([self.model.encode(e) for e in errors], dtype="float32")

        # FAISS-Index anlegen
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

        # Daten auf Festplatte schreiben
        faiss.write_index(self.index, self.index_file)
        with open(self.json_file, "w", encoding="utf-8") as f:
            json.dump(self.error_dict, f, ensure_ascii=False)

        print(f"✅ Neuer Index + JSON in '{self.db_folder}' erstellt.")

    def load_index(self):
        """
        Lädt einen bereits existierenden FAISS-Index und die Fehler-Daten.
        """
        if not (os.path.exists(self.index_file) and os.path.exists(self.json_file)):
            raise FileNotFoundError("❌ Kein FAISS-Index gefunden. Bitte build_index() aufrufen.")

        print("🔎 Lade vorhandenen FAISS-Index...")
        self.index = faiss.read_index(self.index_file)

        with open(self.json_file, "r", encoding="utf-8") as f:
            self.error_dict = json.load(f)

        print("✅ Index & Fehler-Korrekturen geladen.")

    def add_entries(self, new_error_corrections: dict):
        """
        Fügt weitere Fehler-Korrektur-Paare hinzu, ohne alles neu zu bauen.
        """
        if self.index is None:
            # Versuch zu laden, falls vorhanden
            if os.path.exists(self.index_file) and os.path.exists(self.json_file):
                self.load_index()
            else:
                raise FileNotFoundError("❌ Kein Index vorhanden. Bitte erst build_index() nutzen.")

        # Merge in self.error_dict
        for fehler, korrektur in new_error_corrections.items():
            self.error_dict[fehler] = korrektur

        # embeddings nur für die neuen keys
        new_keys = list(new_error_corrections.keys())
        new_embeds = np.array([self.model.encode(k) for k in new_keys], dtype="float32")

        # An Index anhängen
        self.index.add(new_embeds)

        # Speichern
        faiss.write_index(self.index, self.index_file)
        with open(self.json_file, "w", encoding="utf-8") as f:
            json.dump(self.error_dict, f, ensure_ascii=False)

        print(f"✅ {len(new_keys)} neue Einträge hinzugefügt und Index aktualisiert.")

    def query(self, text: str, top_k=3, threshold=0.6):
        """
        Sucht in der DB nach ähnlichen fehlerhaften Formulierungen.

        :param text: Der zu prüfende Satz/Abschnitt
        :param top_k: Anzahl der gesuchten Ähnlichkeiten
        :param threshold: Distanzschwelle
        :return: Liste [(fehler, korrektur), ...]
        """
        if self.index is None:
            self.load_index()

        embed = np.array([self.model.encode(text)], dtype="float32")
        distances, indices = self.index.search(embed, top_k)

        all_errors = list(self.error_dict.keys())

        results = []
        for i in range(top_k):
          idx = indices[0][i]
          # Sicherstellen, dass idx in den Bereich von all_errors passt
          if idx < len(all_errors):
              if distances[0][i] < threshold:
                  fehler_key = all_errors[idx]
                  korrektur = self.error_dict[fehler_key]
                  results.append((fehler_key, korrektur))
        return results


    def retrieve_context(self, seo_text: str) -> str:
        """
        Durchsucht den seo_text Satz für Satz, holt ggf. Korrekturvorschläge
        und baut einen Kontextstring.
        """
        lines = []
        for s in seo_text.split(". "):
            suggestions = self.query(s)
            for old, new in suggestions:
                lines.append(f"- Fehler: {old} ➝ Verbesserung: {new}")

        if lines:
            return "Bekannte Fehler/Korrekturen:\n" + "\n".join(lines)
        else:
            return "Keine bekannten Fehler gefunden."



db = VectorDB(db_folder=FAISS_PATH)
db.build_index(error_corrections)
db.add_entries(new_error_corrections)
#db.retrieve_context(test_text)

In [20]:
%%capture
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from chatbot import Chatbot

# -------------------------
# 2) PromptManager
# -------------------------

class PromptManager:
    """
    Lädt Prompts aus dem /data/prompts Ordner und kombiniert sie mit
    dem Context aus der VectorDB, um einen finalen Prompt zu erstellen.
    """

    def __init__(self, prompts_folder="./data/prompts"):
        """
        :param prompts_folder: Ordner, in dem .json (oder .txt) Prompts liegen
        """
        self.prompts_folder = prompts_folder

    def load_prompt(self, filename: str) -> dict:
        """
        Lädt einen JSON-Prompt aus dem Ordner, z.B. 'grammar_prompt.json'.
        """
        path = os.path.join(self.prompts_folder, filename)
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"⚠️ Prompt-Datei {path} nicht gefunden!")
            return {}
        except json.JSONDecodeError:
            print(f"⚠️ Ungültiges JSON in {path}")
            return {}

    def build_final_prompt(self, base_prompt_file: str, context: str, user_text: str) -> (str, str):
        """
        Kombiniert:
         - base_prompt_file (System-/User-Prompts)
         - den 'context' aus der VectorDB
         - den 'user_text' (SEO-Text)
        und gibt final (system_prompt, user_prompt) zurück.
        """
        prompt_data = self.load_prompt(base_prompt_file)

        system_prompt = prompt_data.get("system_prompt", "")
        user_prompt   = prompt_data.get("user_prompt", "")

        # Kontext an system_prompt anhängen
        system_prompt_full = system_prompt

        # SEO-Text an user_prompt anhängen
        user_prompt_full = user_prompt.format(context=context,optimized_text=user_text)

        return (system_prompt_full, user_prompt_full)

# pm = PromptManager(prompts_folder=PROMPT_PATH)
# context = db.retrieve_context(test_text)
# final_prompts = pm.build_final_prompt("grammar_check.json", context, test_text)

In [21]:

# from chatbot import Chatbot

# -------------------------
# 3) SEOGrammarChecker
# -------------------------

# cb = Chatbot(systemprompt=final_prompts[0], userprompt=final_prompts[1])
# final_text = cb.chat()
# final_text

# 📟SEO Texte multi👅

In [22]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Entferne evtl. vorhandene Handler (optional)
while logger.handlers:
    logger.handlers.pop()

# Füge einen neuen StreamHandler hinzu, der auf stdout schreibt
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(levelname)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# logging.info("Jetzt sollte dieser Text in Colab erscheinen!")


In [23]:
import seopageoptimizer
importlib.reload(seopageoptimizer)
from seopageoptimizer import SEOPageOptimizer

In [24]:
languages_to_produce = ["German"]  # oder nur ["English"]

for url in list(original_texts.keys()):
    for lang in languages_to_produce:
        # SEO-Optimizer erstellen
        optimizer = SEOPageOptimizer(
            output_dir=OUTPUT_PATH + "/final",
            prompts_file=PROMPT_PATH + "/seo_prompts_2.json",
            google_ads_keywords=keywords_llm_google
        )

        url_to_optimize = url
        # Erzeuge sinnvollen Dateinamen, z.B. "page_optimized_English.html"
        sanitized_url = str(url).replace("/", "")
        html_filename = f"{sanitized_url}_optimized_{lang}.html"
        outfile = os.path.join(OUTPUT_PATH, "final", html_filename)


        optimizer.optimize_page(
            url=url_to_optimize,
            outfile=outfile,
            translate=False if lang == "German" else True,
            language=lang
        )

        print(f"✅Done! HTML for {url_to_optimize} in {lang} at:", outfile)


INFO: Lade Prompts aus Datei: /content/drive/MyDrive/Colab Notebooks/SEO/data/prompts/seo_prompts_2.json
INFO: SEO-Optimierung startet => https://www.rue-zahnspange.de/ => /content/drive/MyDrive/Colab Notebooks/SEO/output/final/https:www.rue-zahnspange.de_optimized_German.html
INFO: Lade HTML (raw) von https://www.rue-zahnspange.de/...
INFO: chardet sagt: utf-8 (Confidence: 0.7164597186253236)
INFO: HTML erfolgreich geladen und manuell dekodiert.
INFO: Blocks gesamt: 67
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: ✅ Datei gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/https:www.rue-zahnspange.de_optimized_German.html
INFO: Fertig! => /content/drive/MyDrive/Colab Notebooks/SEO/output/final/https:www.rue-zahnspange.de_optimized_German.html
Report => /

In [25]:
import os
import json

def merge_reports_for_urls(url_list, output_dir=OUTPUT_PATH+"/final", final_merged="final_merged.json"):
    """
    - Liest für jede URL die jeweilige report_{url_sanitized}.json
    - Fasst analysis/improvement/Bloecke zusammen
    - Schreibt am Ende ein großes JSON in `final_merged`
    """
    # Dieses Dictionary wird alle zusammengefassten Infos enthalten
    big_merged_data = {}

    for url in url_list:
        # Erzeuge den Dateinamen analog zur Optimierung
        # z. B.  "report_https:example.compage.html.json"
        sanitized_url = url.replace("/", "")
        report_filename = f"report_{sanitized_url}.json"
        report_path = os.path.join(output_dir, report_filename)

        # Prüfen, ob Datei existiert
        if not os.path.isfile(report_path):
            print(f"⚠️ Report-Datei fehlt: {report_path}")
            continue

        # JSON lesen
        with open(report_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # analysis_of_original_text & improvement_description
        analysis = data.get("analysis_of_original_text", "")
        improvement = data.get("improvement_description", "")

        # Blocks => original_text + optimized_text kumulieren
        blocks = data.get("blocks", [])
        original_texts = []
        optimized_texts = []

        for block in blocks:
            original_texts.append(block.get("original_text",""))
            optimized_texts.append(block.get("optimized_text",""))

        # Zusammenfassen in je einen großen String
        merged_original = "\n".join(original_texts)
        merged_optimized = "\n".join(optimized_texts)

        # In big_merged_data ablegen
        big_merged_data[url] = {
            "analysis": analysis,
            "improvement": improvement,
            "original_text": merged_original,
            "optimized_text": merged_optimized
        }

    # Alles als ein JSON speichern
    final_json_path = os.path.join(output_dir, final_merged)
    with open(final_json_path, "w", encoding="utf-8") as f:
        json.dump(big_merged_data, f, indent=2, ensure_ascii=False)

    print(f"✅ Zusammenfassung erstellt: {final_json_path}")


# Beispiel-Aufruf:
if __name__ == "__main__":
    # Angenommen, hier deine URLs
    urls_to_process = list(original_texts.keys())

    merge_reports_for_urls(urls_to_process, output_dir=OUTPUT_PATH+"/final", final_merged="allinone.json")


✅ Zusammenfassung erstellt: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/allinone.json


In [26]:
import os
import json
import pypandoc

def create_html_report(merged_json_path, out_html="seo_report.html", out_docx="seo_report.docx"):
    """
    Liest das zusammengefasste JSON (alle URLs, Analyse, Verbesserungen, Original & Optimiert)
    und erzeugt daraus ein HTML-Dokument. Anschließend konvertiert es das HTML in eine DOCX-Datei.
    """

    # JSON laden
    with open(merged_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        # => data = {
        #    "https://example.com" : {
        #       "analysis": "...",
        #       "improvement": "...",
        #       "original_text": "...",
        #       "optimized_text": "...",
        #    }, ...
        # }

    # HTML Template zusammenbauen
    # Du kannst hier inline CSS nutzen oder <link> mit externem Stylesheet
    html_head = """
    <!DOCTYPE html>
    <html lang="de">
    <head>
      <meta charset="UTF-8"/>
      <title>SEO Zusammenfassung</title>
      <style>
        body {
          font-family: "Helvetica Neue", Arial, sans-serif;
          margin: 20px;
          line-height: 1.6;
          color: #333;
          background: #f2f2f2;
        }
        h1 {
          text-align: center;
          background: #007BFF;
          color: #fff;
          padding: 10px;
          border-radius: 4px;
        }
        .url-block {
          background: #fff;
          border-radius: 5px;
          padding: 20px;
          margin-bottom: 20px;
          box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .url-block h2 {
          margin-top: 0;
          color: #007BFF;
        }
        .subtitle {
          font-weight: bold;
          color: #555;
          margin-top: 1rem;
        }
        .text-box {
          background: #fafafa;
          border-left: 4px solid #007BFF;
          padding: 10px;
          margin: 10px 0;
          white-space: pre-wrap;  /* Damit \n Zeilenumbrüche anzeigt */
        }
      </style>
    </head>
    <body>
      <h1>SEO Report &amp; Verbesserungs-Übersicht</h1>
    """

    html_body = []

    # Für jede URL Sektion erstellen
    for idx, (url, content) in enumerate(data.items(), start=1):
        analysis = content.get("analysis", "")
        improvement = content.get("improvement", "")
        original_text = content.get("original_text", "")
        optimized_text = content.get("optimized_text", "")

        section_html = f"""
        <div class="url-block">
          <h2>{idx}. {url}</h2>

          <div class="subtitle">Analyse des Originaltexts:</div>
          <div class="text-box">{analysis}</div>

          <div class="subtitle">Erklärung der Verbesserungen:</div>
          <div class="text-box">{improvement}</div>
        </div>
        """
        html_body.append(section_html)

    html_end = """
    </body>
    </html>
    """

    # Finale HTML zusammensetzen
    final_html = html_head + "\n".join(html_body) + html_end

    # Speichern als HTML
    with open(out_html, "w", encoding="utf-8") as f:
        f.write(final_html)

    print(f"✅ HTML-Report erstellt: {out_html}")

    # Mit pypandoc in DOCX konvertieren
    # Dafür muss pypandoc & pandoc installiert sein
    pypandoc.convert_file(
        source_file=out_html,
        to="docx",
        outputfile=out_docx,
        extra_args=["--standalone"]
    )
    print(f"✅ DOCX exportiert: {out_docx}")


if __name__ == "__main__":
    merged_json = OUTPUT_PATH + "/final/allinone.json"  # Pfad zu deinem zusammengefassten JSON
    out_html = OUTPUT_PATH + "/final/seo_report.html"
    out_docx = OUTPUT_PATH + "/final/seo_report.docx"

    create_html_report(merged_json, out_html, out_docx)


✅ HTML-Report erstellt: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/seo_report.html
✅ DOCX exportiert: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/seo_report.docx


# 🔎 SEO Analyses + Statistics

In [30]:
# künstliche SEO-Daten
historical_data = {
    "Date": [
        "2023-01-01", "2023-02-01", "2023-03-01",
        "2023-04-01", "2023-05-01", "2023-06-01"
    ],
    "Organic_Sessions": [200, 220, 250, 400, 450, 480],
    "Conversion_Rate": [0.02, 0.021, 0.022, 0.028, 0.03, 0.031],
    "Average_Time_on_Page": [40, 42, 45, 60, 65, 70]
}

In [31]:
json_path = OUTPUT_PATH + "/final/allinone.json"
with open(json_path, "r", encoding="utf-8") as f:
    seo_json = json.load(f)

In [32]:
import seoanalyzer
importlib.reload(seoanalyzer)
from seoanalyzer import SEOAnalyzer

exclude_list = ["leila", "graf", "koenig", "bjoern", "könig", 'björn', 'adolf', 'schmidt', 'strasse', 'straße', 'tilo', 'remhof']

keywords_final = google_ads_keywords

seo_analyzer = SEOAnalyzer(
    seo_json=seo_json,
    keywords_final=keywords_llm.split(", "),
    output_dir=OUTPUT_PATH+"/images",
    historical_data=historical_data,
    wordcloud_exclude=exclude_list,
    shared_image_dict=FINAL_IMAGES
)

analysis_paths = seo_analyzer.run_analysis()
print("Analysis Plot Pfade:", analysis_paths)

model_paths = seo_analyzer.run_models()
print("Model Plot Pfade:", model_paths)

all_paths = seo_analyzer.get_all_image_paths()
print("Alle Pfade gesammelt:", all_paths)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🔍 Ähnlichkeitsanalyse gestartet...
📊 Similarities (Original -> Keywords): [0.04244005 0.14526635 0.10848612 0.11971188 0.12999408 0.15360336
 0.10074743 0.09759751 0.11161668 0.02787655 0.00726353 0.04454286
 0.         0.09051633 0.05755932 0.05377227 0.04468247 0.04412238
 0.11303598 0.09234653 0.1233016  0.04271167 0.08214456 0.04244005
 0.0858948  0.02493042 0.10701081 0.01450754]
📊 Similarities (Optimiert -> Keywords): [0.10411979 0.16770985 0.13861343 0.15624783 0.15394831 0.17195114
 0.12699301 0.13720301 0.14395178 0.05377802 0.04421889 0.0464174
 0.02415488 0.10681165 0.07050703 0.0971802  0.05652891 0.07419851
 0.11815939 0.11765007 0.14368003 0.06867522 0.09245407 0.10411979
 0.12274948 0.06321405 0.14179104 0.03077733]
📊 Visualisierung der Similarities:
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/similarity_scores_2025.png
🌍 Wordcloud-Visualisierung:
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_filt

<Figure size 640x480 with 0 Axes>

✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/session_forecast_2025.png
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/conversion_forecast_2025.png
Model Plot Pfade: {'seo_trends': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/seo_trends_2025.png', 'session_forecast': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/session_forecast_2025.png', 'conversion_forecast': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/conversion_forecast_2025.png'}
Alle Pfade gesammelt: {'keyword_heatmap': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png', 'embedding_keyword_3d': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/embedding_keyword_3d_2025.png', 'cosine_similarity_steps': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_similarity_steps_2025.png', 'cosine_comparison_3d': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_comparison_3d_2025

#🥼Lab